# Clustering

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Load the data

In [ ]:
data_path = './data/clustering'